# Macaw OpenVoice — API Validation Notebook

Validates **all REST API endpoints and WebSocket protocol** of Macaw OpenVoice.
Run all cells sequentially on Google Colab (GPU runtime recommended).

**What this validates:**
- Health and system endpoints
- Audio transcription (STT) — all formats, language, word timestamps
- Audio translation
- Speech synthesis (TTS) — WAV, PCM, effects, alignment, seed
- Voice management — CRUD operations
- WebSocket realtime — STT streaming, TTS full-duplex
- Error handling — expected error responses

> **Pre-requisite:** Select a GPU runtime (Runtime > Change runtime type > T4 GPU)

## 1. Setup

In [1]:
# Edit these before running
STT_MODEL = 'faster-whisper-tiny'
TTS_MODEL = 'kokoro-v1'
SERVER_PORT = 8000
BASE_URL = f'http://localhost:{SERVER_PORT}'

# Installation source:
#   'pip'     — install released version from PyPI
#   'develop' — install from git develop branch (pre-release validation)
INSTALL_FROM = 'develop'

### Installation

Installs from **PyPI** or **git develop branch** based on `INSTALL_FROM` above.

> **Note (Colab):** You may see a pip resolver warning about `protobuf` conflicts with pre-installed `tensorflow`/`grpcio-status`. This is harmless — Macaw requires `protobuf>=6.31` and does not use TensorFlow. The warning can be safely ignored.

In [3]:
%%capture
if INSTALL_FROM == 'develop':
    !pip install 'macaw-openvoice[faster-whisper,kokoro,itn,codec] @ git+https://github.com/usemacaw/macaw-openvoice.git@develop'
else:
    !pip install macaw-openvoice[faster-whisper,kokoro,itn,codec]

In [4]:
%%capture
!macaw pull $STT_MODEL
!macaw pull $TTS_MODEL

### Start server

In [5]:
import os
os.environ['MACAW_WORKER_HEALTH_PROBE_TIMEOUT_S'] = '300'
os.environ['MACAW_LOG_FORMAT'] = 'json'  # avoid ANSI escape codes in logs
os.environ['MACAW_VOICE_DIR'] = '/tmp/macaw_voices'

In [6]:
!nohup macaw serve --host 0.0.0.0 --port $SERVER_PORT --voice-dir /tmp/macaw_voices > /tmp/macaw.log 2>&1 &

In [7]:
%cat /tmp/macaw.log

{"component": "registry", "name": "faster-whisper-tiny", "engine": "faster-whisper", "event": "model_found", "level": "info", "timestamp": "2026-02-20T19:25:59.576062Z"}
{"component": "registry", "name": "kokoro-v1", "engine": "kokoro", "event": "model_found", "level": "info", "timestamp": "2026-02-20T19:25:59.577917Z"}
{"component": "worker.manager", "worker_id": "faster-whisper-50051", "port": 50051, "engine": "faster-whisper", "worker_type": "stt", "event": "spawning_worker", "level": "info", "timestamp": "2026-02-20T19:25:59.578274Z"}
{"component": "cli.serve", "model": "faster-whisper-tiny", "engine": "faster-whisper", "port": 50051, "worker_type": "stt", "event": "worker_spawned", "level": "info", "timestamp": "2026-02-20T19:25:59.579174Z"}
{"component": "worker.manager", "worker_id": "kokoro-50052", "port": 50052, "engine": "kokoro", "worker_type": "tts", "event": "spawning_worker", "level": "info", "timestamp": "2026-02-20T19:25:59.579563Z"}
{"component": "cli.serve", "model": 

In [8]:
import time, httpx

print('Waiting for server ...')
for i in range(90):
    try:
        r = httpx.get(f'{BASE_URL}/health', timeout=5)
        if r.json().get('status') == 'ok':
            print(f'Server ready! (attempt {i+1})')
            print(r.json())
            break
    except Exception:
        pass
    time.sleep(2)
else:
    print('Server did not start. Logs:')
    !tail -50 /tmp/macaw.log
    raise RuntimeError('Server not ready')

Waiting for server ...
Server ready! (attempt 13)
{'status': 'ok', 'version': '0.1.7', 'models_loaded': 2, 'workers_ready': 2, 'workers_total': 2}


## 2. Health & System Endpoints

### 2.1 `GET /health`

In [9]:
import httpx

r = httpx.get(f'{BASE_URL}/health', timeout=10)
data = r.json()
print(data)
assert r.status_code == 200
assert data['status'] == 'ok'
assert data['workers_ready'] > 0

{'status': 'ok', 'version': '0.1.7', 'models_loaded': 2, 'workers_ready': 2, 'workers_total': 2}


### 2.2 `GET /v1/models`

In [10]:
r = httpx.get(f'{BASE_URL}/v1/models', timeout=10)
data = r.json()
print(data)
assert data['object'] == 'list'
model_ids = [m['id'] for m in data['data']]
assert STT_MODEL in model_ids, f'{STT_MODEL} not loaded'
assert TTS_MODEL in model_ids, f'{TTS_MODEL} not loaded'

{'object': 'list', 'data': [{'id': 'faster-whisper-tiny', 'object': 'model', 'owned_by': 'macaw', 'created': 0, 'type': 'stt', 'engine': 'faster-whisper'}, {'id': 'kokoro-v1', 'object': 'model', 'owned_by': 'macaw', 'created': 0, 'type': 'tts', 'engine': 'kokoro'}]}


## 3. Generate Test Audio

Uses TTS to generate speech, then uses that audio for STT tests (round-trip).

In [11]:
TEST_TEXT = 'Hello world. This is a test of the Macaw voice system.'

r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': TEST_TEXT, 'voice': 'default'},
    timeout=120,
)
assert r.status_code == 200, f'TTS failed: {r.status_code} {r.text}'
assert len(r.content) > 1000

TEST_AUDIO = '/tmp/test_audio.wav'
with open(TEST_AUDIO, 'wb') as f:
    f.write(r.content)
print(f'Test audio: {len(r.content):,} bytes')

Test audio: 187,244 bytes


## 4. Audio Transcription (`POST /v1/audio/transcriptions`)

### 4.1 JSON format

In [12]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'json'},
        timeout=120,
    )
data = r.json()
print(data)
assert r.status_code == 200
assert len(data['text'].strip()) > 0

{'text': 'Hello world, this is a test of the Macau voice system.'}


### 4.2 Verbose JSON format

In [13]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'verbose_json'},
        timeout=120,
    )
data = r.json()
print(f"Language={data['language']}, Duration={data['duration']}s")
print(f"Text: {data['text']}")
print(f"Segments: {len(data['segments'])}")
assert 'segments' in data
assert 'language' in data
assert 'duration' in data

Language=en, Duration=3.9013750553131104s
Text: Hello world, this is a test of the Macau voice system.
Segments: 1


### 4.3 Text format

In [14]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'text'},
        timeout=120,
    )
print(repr(r.text))
assert r.status_code == 200
assert len(r.text.strip()) > 0

'Hello world, this is a test of the Macau voice system.'


### 4.4 SRT format

In [15]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'srt'},
        timeout=120,
    )
print(r.text)
assert '-->' in r.text, 'No SRT timestamps'

1
00:00:00,000 --> 00:00:04,000
Hello world, this is a test of the Macau voice system.



### 4.5 VTT format

In [16]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'vtt'},
        timeout=120,
    )
print(r.text)
assert 'WEBVTT' in r.text

WEBVTT

00:00:00.000 --> 00:00:04.000
Hello world, this is a test of the Macau voice system.



### 4.6 With explicit language

In [17]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'language': 'en', 'response_format': 'verbose_json'},
        timeout=120,
    )
data = r.json()
print(f"Language: {data['language']}, Text: {data['text']}")
assert data['language'] == 'en'

Language: en, Text: Hello world, this is a test of the Macau voice system.


### 4.7 With word-level timestamps

In [18]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/transcriptions',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL, 'response_format': 'verbose_json',
              'timestamp_granularities[]': 'word'},
        timeout=120,
    )
data = r.json()
words = data.get('words', [])
print(f'Words: {words[:5]}')
assert len(words) > 0, 'No word timestamps'
assert 'start' in words[0]
assert 'end' in words[0]

Words: [{'word': 'Hello', 'start': 0.0, 'end': 0.6399999856948853}, {'word': 'world,', 'start': 0.6399999856948853, 'end': 1.059999942779541}, {'word': 'this', 'start': 1.4600000381469727, 'end': 1.659999966621399}, {'word': 'is', 'start': 1.659999966621399, 'end': 1.8600000143051147}, {'word': 'a', 'start': 1.8600000143051147, 'end': 1.9800000190734863}]


## 5. Audio Translation (`POST /v1/audio/translations`)

In [19]:
with open(TEST_AUDIO, 'rb') as f:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/translations',
        files={'file': ('test.wav', f, 'audio/wav')},
        data={'model': STT_MODEL},
        timeout=120,
    )
data = r.json()
print(data)
assert r.status_code == 200
assert len(data['text'].strip()) > 0

{'text': 'Hello world, this is a test of the Macau voice system.'}


## 6. Speech Synthesis (`POST /v1/audio/speech`)

### 6.1 WAV format (default)

In [20]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Hello world', 'voice': 'default'},
    timeout=120,
)
assert r.status_code == 200
assert r.content[:4] == b'RIFF', 'Not a WAV file'
print(f'WAV: {len(r.content):,} bytes')

WAV: 73,244 bytes


### 6.2 PCM format

In [21]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Hello world', 'response_format': 'pcm'},
    timeout=120,
)
assert r.status_code == 200
assert r.content[:4] != b'RIFF', 'Should be raw PCM, not WAV'
assert len(r.content) % 2 == 0, 'PCM 16-bit must be even bytes'
print(f'PCM: {len(r.content):,} bytes')

PCM: 73,200 bytes


### 6.3 Speed control

In [22]:
r_normal = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Testing speed control', 'speed': 1.0},
    timeout=120,
)
r_fast = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Testing speed control', 'speed': 2.0},
    timeout=120,
)
print(f'Normal: {len(r_normal.content):,} bytes')
print(f'Fast:   {len(r_fast.content):,} bytes')
assert len(r_fast.content) < len(r_normal.content), 'Fast should be shorter'

Normal: 90,044 bytes
Fast:   45,644 bytes


### 6.4 Audio effects — pitch shift

In [23]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Pitch shift test',
          'effects': {'pitch_shift_semitones': 3.0}},
    timeout=120,
)
assert r.status_code == 200
print(f'Pitch shift: {len(r.content):,} bytes')

Pitch shift: 82,844 bytes


### 6.5 Audio effects — reverb

In [24]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Reverb test',
          'effects': {'reverb_room_size': 0.7, 'reverb_damping': 0.5,
                      'reverb_wet_dry_mix': 0.3}},
    timeout=120,
)
assert r.status_code == 200
print(f'Reverb: {len(r.content):,} bytes')

Reverb: 85,244 bytes


### 6.6 Audio effects — combined

In [25]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Combined effects',
          'effects': {'pitch_shift_semitones': -2.0,
                      'reverb_room_size': 0.5, 'reverb_wet_dry_mix': 0.2}},
    timeout=120,
)
assert r.status_code == 200
print(f'Combined: {len(r.content):,} bytes')

Combined: 81,644 bytes


### 6.7 Word-level alignment (NDJSON)

In [26]:
import json as _json

r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Hello world',
          'include_alignment': True, 'alignment_granularity': 'word'},
    timeout=120,
)
assert r.status_code == 200
assert 'ndjson' in r.headers.get('content-type', '')

lines = [_json.loads(l) for l in r.text.strip().split('\n') if l.strip()]
audio_lines = [l for l in lines if l['type'] == 'audio']
done_lines = [l for l in lines if l['type'] == 'done']
print(f'Audio chunks: {len(audio_lines)}, Done: {len(done_lines)}')

aligned = [l for l in audio_lines if l.get('alignment')]
if aligned:
    print(f"Alignment: {aligned[0]['alignment']}")
assert len(done_lines) == 1

Audio chunks: 18, Done: 1
Alignment: {'items': [{'text': 'Hello', 'start_ms': 350, 'duration_ms': 275}, {'text': 'world', 'start_ms': 625, 'duration_ms': 625}], 'granularity': 'word'}


### 6.8 Character-level alignment

In [27]:
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Hi',
          'include_alignment': True, 'alignment_granularity': 'character'},
    timeout=120,
)
assert r.status_code == 200
lines = [_json.loads(l) for l in r.text.strip().split('\n') if l.strip()]
aligned = [l for l in lines if l.get('type') == 'audio' and l.get('alignment')]
if aligned:
    a = aligned[0]['alignment']
    print(f"Granularity: {a.get('granularity')}, Items: {a['items']}")
    assert a['granularity'] == 'character'

Granularity: character, Items: [{'text': 'H', 'start_ms': 375, 'duration_ms': 262}, {'text': 'i', 'start_ms': 637, 'duration_ms': 263}]


### 6.9 Seed parameter

> **Note:** Kokoro is a deterministic engine — `seed` is accepted but has no effect. Seed is meaningful for non-deterministic engines like Qwen3-TTS where it controls sampling behavior.

In [28]:
payload = {'model': TTS_MODEL, 'input': 'Reproducibility test', 'seed': 42}
r1 = httpx.post(f'{BASE_URL}/v1/audio/speech', json=payload, timeout=120)
assert r1.status_code == 200
print(f'Seed accepted. Size: {len(r1.content):,} bytes')

Seed accepted. Size: 98,444 bytes


### 6.10 Text normalization

In [29]:
for mode in ['auto', 'on', 'off']:
    r = httpx.post(
        f'{BASE_URL}/v1/audio/speech',
        json={'model': TTS_MODEL, 'input': 'I have 3 cats.',
              'text_normalization': mode},
        timeout=120,
    )
    assert r.status_code == 200
    print(f'{mode}: {len(r.content):,} bytes')

auto: 86,444 bytes
on: 86,444 bytes
off: 86,444 bytes


## 7. Voice Management

### 7.1 List preset voices

In [30]:
r = httpx.get(f'{BASE_URL}/v1/voices', timeout=30)
data = r.json()
assert data['object'] == 'list'
assert len(data['data']) > 0
print(f"Total voices: {len(data['data'])}")
for v in data['data'][:5]:
    print(f"  {v['voice_id']}: {v['name']} ({v.get('language')})")

Total voices: 54
  af_alloy: af_alloy (en)
  af_aoede: af_aoede (en)
  af_bella: af_bella (en)
  af_heart: af_heart (en)
  af_jessica: af_jessica (en)


### 7.2 Voice CRUD (create / get / use / delete)

In [31]:
# Create a saved voice
r = httpx.post(
    f'{BASE_URL}/v1/voices',
    data={'name': 'test-voice', 'voice_type': 'designed',
          'instruction': 'A calm and warm English voice', 'language': 'en'},
    timeout=30,
)
assert r.status_code == 201, f'Create voice failed: {r.status_code} {r.text}'
VOICE_ID = r.json()['voice_id']
print(f'Created voice: {VOICE_ID}')

Created voice: 333742fe-336a-4858-8b67-f1fced51d0d1


In [32]:
# Get saved voice
r = httpx.get(f'{BASE_URL}/v1/voices/{VOICE_ID}', timeout=10)
assert r.status_code == 200
print(f"Get voice: {r.json()['voice_id']}, name={r.json()['name']}")

# Use saved voice in synthesis
r = httpx.post(
    f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'Saved voice test',
          'voice': f'voice_{VOICE_ID}'},
    timeout=120,
)
assert r.status_code == 200
print(f'Audio with saved voice: {len(r.content):,} bytes')

# Delete saved voice
r = httpx.delete(f'{BASE_URL}/v1/voices/{VOICE_ID}', timeout=10)
assert r.status_code == 204
r2 = httpx.get(f'{BASE_URL}/v1/voices/{VOICE_ID}', timeout=10)
assert r2.status_code == 404
print('Voice deleted and confirmed gone.')

Get voice: 333742fe-336a-4858-8b67-f1fced51d0d1, name=test-voice
Audio with saved voice: 88,844 bytes
Voice deleted and confirmed gone.


## 8. WebSocket Realtime (`/v1/realtime`)

Tests bidirectional WebSocket protocol for STT streaming and TTS full-duplex.

In [33]:
%%capture
!pip install websockets

### 8.1 STT streaming

In [34]:
import asyncio, json, websockets

async def test_ws_stt():
    ws_url = f'ws://localhost:{SERVER_PORT}/v1/realtime?model={STT_MODEL}'
    async with websockets.connect(ws_url) as ws:
        msg = json.loads(await asyncio.wait_for(ws.recv(), timeout=10))
        assert msg['type'] == 'session.created'
        print(f"Session: {msg['session_id']}")

        # Send test audio PCM (skip 44-byte WAV header)
        with open(TEST_AUDIO, 'rb') as f:
            pcm = f.read()[44:]
        for i in range(0, min(len(pcm), 48000), 3200):
            await ws.send(pcm[i:i+3200])
            await asyncio.sleep(0.05)

        await ws.send(json.dumps({'type': 'input_audio_buffer.commit'}))

        events = []
        try:
            while True:
                raw = await asyncio.wait_for(ws.recv(), timeout=15)
                if isinstance(raw, str):
                    ev = json.loads(raw)
                    events.append(ev)
                    print(f"  {ev['type']}")
                    if ev['type'] == 'transcript.final':
                        print(f"  Text: {ev['text']}")
                        break
        except asyncio.TimeoutError:
            pass

        types = [e['type'] for e in events]
        assert 'transcript.final' in types or 'transcript.partial' in types
        await ws.send(json.dumps({'type': 'session.close'}))

await test_ws_stt()

Session: sess_fa87d9875e6e
  vad.speech_start
  transcript.final
  Text: No, no, no.


### 8.2 Session configuration

In [35]:
async def test_ws_configure():
    ws_url = f'ws://localhost:{SERVER_PORT}/v1/realtime?model={STT_MODEL}'
    async with websockets.connect(ws_url) as ws:
        msg = json.loads(await asyncio.wait_for(ws.recv(), timeout=10))
        assert msg['type'] == 'session.created'

        await ws.send(json.dumps({
            'type': 'session.configure',
            'language': 'en',
            'enable_partial_transcripts': True,
            'vad_sensitivity': 'high',
        }))
        print('Configure sent OK')

        await ws.send(json.dumps({'type': 'session.close'}))
        try:
            while True:
                await asyncio.wait_for(ws.recv(), timeout=3)
        except (asyncio.TimeoutError, websockets.exceptions.ConnectionClosed):
            pass
        print('Session closed OK')

await test_ws_configure()

Configure sent OK
Session closed OK


### 8.3 TTS via WebSocket

In [36]:
async def test_ws_tts():
    ws_url = f'ws://localhost:{SERVER_PORT}/v1/realtime?model={STT_MODEL}'
    async with websockets.connect(ws_url) as ws:
        json.loads(await asyncio.wait_for(ws.recv(), timeout=10))

        await ws.send(json.dumps({'type': 'session.configure', 'model_tts': TTS_MODEL}))
        await ws.send(json.dumps({'type': 'tts.speak', 'text': 'Hello from WebSocket',
                                  'request_id': 'test_tts_1'}))

        events, audio_frames = [], []
        try:
            while True:
                raw = await asyncio.wait_for(ws.recv(), timeout=30)
                if isinstance(raw, bytes):
                    audio_frames.append(raw)
                else:
                    ev = json.loads(raw)
                    events.append(ev)
                    print(f"  {ev['type']}")
                    if ev['type'] == 'tts.speaking_end':
                        break
        except asyncio.TimeoutError:
            pass

        types = [e['type'] for e in events]
        assert 'tts.speaking_start' in types
        assert 'tts.speaking_end' in types
        total = sum(len(f) for f in audio_frames)
        print(f'Audio: {len(audio_frames)} frames, {total:,} bytes')
        assert total > 500
        await ws.send(json.dumps({'type': 'session.close'}))

await test_ws_tts()

  tts.speaking_start
  tts.speaking_end
Audio: 23 frames, 91,200 bytes


### 8.4 TTS with alignment

In [37]:
async def test_ws_alignment():
    ws_url = f'ws://localhost:{SERVER_PORT}/v1/realtime?model={STT_MODEL}'
    async with websockets.connect(ws_url) as ws:
        json.loads(await asyncio.wait_for(ws.recv(), timeout=10))
        await ws.send(json.dumps({'type': 'session.configure', 'model_tts': TTS_MODEL}))
        await ws.send(json.dumps({'type': 'tts.speak', 'text': 'Alignment test',
                                  'include_alignment': True, 'request_id': 'align'}))

        events = []
        try:
            while True:
                raw = await asyncio.wait_for(ws.recv(), timeout=30)
                if isinstance(raw, str):
                    ev = json.loads(raw)
                    events.append(ev)
                    if ev['type'] == 'tts.speaking_end':
                        break
        except asyncio.TimeoutError:
            pass

        types = [e['type'] for e in events]
        assert 'tts.alignment' in types, f'No alignment event. Got: {types}'
        ae = next(e for e in events if e['type'] == 'tts.alignment')
        print(f"Alignment items: {ae['items']}")
        await ws.send(json.dumps({'type': 'session.close'}))

await test_ws_alignment()

Alignment items: [{'text': 'Alignment', 'start_ms': 350, 'duration_ms': 425}, {'text': 'test', 'start_ms': 775, 'duration_ms': 575}]


### 8.5 TTS cancel

In [38]:
async def test_ws_cancel():
    ws_url = f'ws://localhost:{SERVER_PORT}/v1/realtime?model={STT_MODEL}'
    async with websockets.connect(ws_url) as ws:
        json.loads(await asyncio.wait_for(ws.recv(), timeout=10))
        await ws.send(json.dumps({'type': 'session.configure', 'model_tts': TTS_MODEL}))
        await ws.send(json.dumps({'type': 'tts.speak', 'request_id': 'cancel_test',
            'text': 'This long sentence should be cancelled before finishing.'}))

        # Wait for speaking_start, then cancel
        try:
            while True:
                raw = await asyncio.wait_for(ws.recv(), timeout=10)
                if isinstance(raw, str):
                    ev = json.loads(raw)
                    if ev['type'] == 'tts.speaking_start':
                        await ws.send(json.dumps({'type': 'tts.cancel'}))
                        break
        except asyncio.TimeoutError:
            pass

        # Wait for cancelled speaking_end
        cancelled = False
        try:
            while True:
                raw = await asyncio.wait_for(ws.recv(), timeout=10)
                if isinstance(raw, str):
                    ev = json.loads(raw)
                    if ev['type'] == 'tts.speaking_end':
                        cancelled = ev.get('cancelled', False)
                        break
        except asyncio.TimeoutError:
            pass

        print(f'Cancelled: {cancelled}')
        assert cancelled
        await ws.send(json.dumps({'type': 'session.close'}))

await test_ws_cancel()

Cancelled: True


## 9. Error Handling

In [39]:
# Empty text
r = httpx.post(f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': ''}, timeout=30)
assert r.status_code >= 400
print(f'Empty text: {r.status_code}')

# Non-existent model
r = httpx.post(f'{BASE_URL}/v1/audio/speech',
    json={'model': 'nonexistent', 'input': 'test'}, timeout=30)
assert r.status_code >= 400
print(f'Bad model: {r.status_code}')

# Non-existent voice
r = httpx.get(f'{BASE_URL}/v1/voices/nonexistent', timeout=10)
assert r.status_code == 404
print(f'Bad voice: {r.status_code}')

# Invalid audio
r = httpx.post(f'{BASE_URL}/v1/audio/transcriptions',
    files={'file': ('bad.txt', b'not audio', 'text/plain')},
    data={'model': STT_MODEL}, timeout=30)
assert r.status_code == 400
print(f'Bad audio: {r.status_code}')

# Alignment + opus conflict
r = httpx.post(f'{BASE_URL}/v1/audio/speech',
    json={'model': TTS_MODEL, 'input': 'test',
          'include_alignment': True, 'response_format': 'opus'}, timeout=30)
assert r.status_code == 400
print(f'Alignment+opus: {r.status_code}')

Empty text: 422
Bad model: 404
Bad voice: 404
Bad audio: 400
Alignment+opus: 400


## 10. Cleanup

In [40]:
import subprocess
subprocess.run(['pkill', '-f', 'macaw serve'],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print('Server stopped.')

Server stopped.
